In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import momepy as mm

In [ ]:
region_id = 69333
val_path = '/data/uscuni-ulce/validation/val_morphotopes/prague_morphotopes.pq'

# region_id = 5883
# val_path = '../data/freiburg_morphotopes.pq'

In [3]:

etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
tess = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet')
streets = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/streets_chars_{region_id}.parquet')
nodes = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/nodes_chars_{region_id}.parquet')
primary = pd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/primary_chars_{region_id}.parquet')

In [8]:
### p[lot existing data
from core.cluster_validation import generate_detailed_clusters
morphotopes = generate_detailed_clusters(etcs, path = val_path)
morphotope_etcs = primary.loc[morphotopes.index]
morphotope_etcs['mlabel'] = morphotopes.values

types = pd.Series([0,0, 1,1, 2,2,2,2, 3,3,  4,4], index=[134,14, 18,31, 7,5,37,36, 57,58, 30,28])
types = types.replace({0: 'houses', 1: 'grid', 2: 'apartment blocks', 3: 'industry', 4: 'medieval'})
morphotope_etcs = morphotope_etcs[morphotope_etcs['mlabel'].isin(types.index)]
morphotope_etcs['types'] = morphotope_etcs['mlabel'].map(lambda x: types.loc[x])

In [9]:
# morphotope_etcs['types'] = morphotope_etcs['mlabel']
morphotopes = generate_detailed_clusters(etcs, path = val_path)
morphotope_buildings = buildings.loc[morphotopes.index]
morphotope_buildings['mlabel'] = morphotopes.values

In [13]:
morphotope_buildings.explore(column='types', prefer_canvas=True, tiles='cartodbpositron')

(15797, 37)

In [56]:
res = morphotope_etcs.groupby('types').mean().loc[['industry', 'apartment blocks', 'medieval', 'grid', 'houses'],]
res.style.background_gradient(axis=0, cmap="BuGn")

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,ssbCCM,ssbCCD,stbOri,mtbSWR,libNCo,ldbPWL,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbCeA,stbSAl,sdsLen,sssLin,ldsMSL,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,sddAre,midRea,midAre,stcOri,sdcLAL,sdcAre,sscCCo,sscERI,mtcWNe,mdcAre,ltcWRB,sicCAR,stcSAl,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB,sdsAre,likWCe,mibCou,mibAre,mibLen,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,licBAD,misBAD,midBAD,mlabel
types,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
industry,1270.474295,127.329200,2.276912,0.467135,8.682749,2.728936,0.941211,0.563639,19.621032,2.979760,6.031368,0.117932,0.070175,242.908713,0.838232,3.145336,31.309288,34.783324,7.190014,10.553302,537.418533,0.935800,267.000175,210.741228,1154074.325478,0.073120,34.826350,0.796097,3.419596,2.628655,0.105618,0.642447,0.172064,0.177585,0.000001,0.005442,0.006258,926.096475,0.031802,421.000669,134511.390997,41.957602,234360.432613,10.543640,109.023914,5851.280838,0.452353,0.962727,0.031858,48983.892150,0.000016,0.195775,11.709360,826314.297550,5711.610316,0.441706,0.685752,1340.808982,2.233550,0.003292,0.216762,159851.466883,0.000259,3.049708,3576.865786,246.319994,0.555495,0.875067,0.434912,75.732449,7.275404,7.275404,2634.168230,3059.236459,2294.549934,2254.946187,57.739766
apartment blocks,1512.320135,230.072690,23.330670,0.382571,11.781163,3.675896,0.799591,0.533170,21.931058,6.982573,15.491799,0.009326,0.049861,242.189370,0.965001,4.188986,45.760935,48.352784,7.939629,8.769447,242.522191,0.886234,153.868545,39.590028,341822.428554,0.018076,39.472413,0.732313,2.724182,2.927978,0.064547,0.619279,0.164531,0.210517,0.000001,0.006453,0.010932,354.245208,0.020083,178.029130,31677.186528,6.925208,65843.791085,18.537336,157.069137,10325.228161,0.493659,0.954403,0.018388,76906.217006,0.000024,0.157676,9.855349,288273.038647,3814.141722,0.405534,0.706789,866.440491,22.207160,0.004715,0.152121,32975.535645,0.000132,1.124654,1654.814395,246.186989,0.519682,0.794395,0.367826,79.297420,6.210523,6.210523,1407.954667,1643.761618,630.562337,542.932870,17.415512
medieval,480.659535,118.202184,36.899494,0.432754,14.532468,11.022089,0.798696,0.617701,13.613144,3.547446,13.977451,0.411331,16.663781,1061.549141,0.408502,6.302571,6.929225,8.325369,5.362424,8.983583,143.206962,0.951658,91.981592,109.959596,148082.097733,0.078891,14.033304,0.188461,3.639196,2.898990,0.102970,0.773054,0.081416,0.145360,0.000002,0.009701,0.017183,226.193533,0.018446,105.452352,12048.794105,23.515152,32819.021320,15.889131,51.110149,1101.443879,0.433576,0.916013,0.059241,9664.658397,0.000106,0.547249,9.290617,307190.554735,3323.190192,0.448315,0.774705,582.150791,15.870697,0.009841,0.411300,13369.763153,0.000958,26.340548,12540.409313,1868.577891,0.550875,0.366918,0.307754,221.051500,6.479401,6.479401,521.189245,724.441290,375.222171,389.345511,29.269841
grid,403.484888,83.810831,1.479552,0.564594,9.642303,5.633954,0.928546,0.801177,11.436599,2.307121,9.550870,0.376023,1.771872,415.744632,0.411287,1.902642,12.908744,15.159673,2.450193,2.709254,120.297905,0.988158,109.072207,217.331118,235975.240871,0.070276,21.173255,0.272137,1.751505,3.558140,0.224207,0.512626,0.427171,0.049464,0.000002,0.006404,0.015009,34.944652,0.083570,108.319923,11902.241226,39.629014,39555.227234,10.359242,50.110645,1189.958093,0.476944,0.965663,0.060119,11058.181465,0.000138,0.424163,3.476821,39220.319985,791.646352,0.533433,0.941260,132.911236,10.295062,0.016634,0.418728,8983.277284,0.001245,14.326689,5264.407484,730.328816,0.639820,0.455948,0.362131,133.480713,6.816436,6.816436,413.879422,643.952177,175.747910,298.189909,25.874862
houses,205.579261,56.868680,5.424916,0.565515,10.381982,5.404923,0.933378,0.777447,7.626204,1.263540,16.120846,0.106068,0.028829,80.780617,0.853885,3.108341,18.487486,19.587550,5.473027,4.156587,178.129896,0.960602,125.748393,237.144144,277388.116199,0.

In [ ]:
# differences between apartment blocks and industrial -
# ssbEri (more rectangular), ssbCCD ( more complex patterns),
# sdsLen ( simpler streets), ldsMSL(longer, simpler streets), ldsRea(more buildings), ldsAre(more built up area),
# ldsCDL(simpler streets), mtdMDi(simpler streets), sddAre(more built up area), midRea(more buildings), sdsAre(more built up area)
# stds
# sdbAre(less regular patterns), sssLin(less complex streets), ldsAre (more built up), ldsCDL(less regular streets), sddAre	midRea	midAre sdsAre mibFR

In [58]:
res = morphotope_etcs.groupby('types').std().loc[['industry', 'apartment blocks', 'medieval', 'grid', 'houses'],]
res.style.background_gradient(axis=0, cmap="BuGn")

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,ssbCCM,ssbCCD,stbOri,mtbSWR,libNCo,ldbPWL,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbCeA,stbSAl,sdsLen,sssLin,ldsMSL,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,sddAre,midRea,midAre,stcOri,sdcLAL,sdcAre,sscCCo,sscERI,mtcWNe,mdcAre,ltcWRB,sicCAR,stcSAl,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB,sdsAre,likWCe,mibCou,mibAre,mibLen,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,licBAD,misBAD,midBAD,mlabel
types,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
industry,2900.028019,129.499762,48.105007,0.135983,7.867139,8.161712,0.098227,0.231144,18.137974,4.954104,9.795935,0.202840,0.321580,294.572529,0.192184,4.900308,24.433677,10.489403,10.512778,12.757850,311.967266,0.100632,82.503678,75.614198,707891.958320,0.086579,9.333892,0.176278,1.515956,1.186601,0.036161,0.091189,0.043659,0.095879,0.000000,0.020017,0.001797,637.851453,0.046042,235.698154,88563.006927,26.755270,174383.442507,12.048615,67.639534,7786.325789,0.144515,0.070558,0.020249,37897.225090,0.000015,0.175163,11.927243,449842.377489,2697.998434,0.120062,0.156661,735.133139,4.222630,0.002054,0.050579,133649.376368,0.000312,4.718784,7465.508604,303.038035,0.241048,0.163492,0.146366,79.874246,7.319347,7.319347,3058.345179,1498.163474,1656.100582,1768.231484,0.439083
apartment blocks,1645.178723,183.063607,209.743999,0.152399,14.736926,9.493299,0.184989,0.259255,16.377615,6.446460,7.595403,0.032549,0.303235,177.613831,0.080688,5.038726,24.379546,11.780535,8.875431,9.780044,348.894160,0.210395,46.749819,25.823080,214216.934080,0.018466,8.131071,0.208947,1.662163,0.875797,0.044644,0.091462,0.059647,0.082429,0.000000,0.002556,0.002511,244.825084,0.043000,139.377504,27633.759843,5.340875,52927.280405,10.873722,67.477794,8935.340788,0.110045,0.076301,0.006101,41047.036977,0.000009,0.084170,9.735064,278240.689110,3316.201488,0.137334,0.237093,784.873450,12.359432,0.002397,0.059606,37579.114893,0.000067,0.377825,1679.311295,187.667228,0.265345,0.184609,0.158166,48.867160,2.500949,2.500949,1243.043651,693.158772,975.143301,1061.765578,15.017922
medieval,521.409465,85.095884,120.777302,0.115007,13.229003,8.199241,0.161385,0.213733,6.603273,2.859820,11.785154,0.220806,14.690480,939.357278,0.160454,4.195119,7.585712,4.197006,7.489819,9.237171,110.664533,0.101124,24.664323,36.237553,81724.307139,0.038277,5.753660,0.149847,1.686737,0.639420,0.037893,0.091212,0.056664,0.110634,0.000001,0.002347,0.004186,359.655021,0.037049,60.481661,10774.464379,13.165635,39939.664033,13.048065,27.399784,1421.370176,0.123256,0.078275,0.021421,7744.956170,0.000061,0.192907,9.539156,495399.941494,4717.819448,0.107316,0.226994,722.110333,12.943077,0.005780,0.216910,14064.651515,0.000625,22.157725,11354.121072,1764.033968,0.184154,0.205803,0.123422,130.080485,1.677299,1.677299,368.621712,472.868557,294.812913,296.831089,0.963600
grid,1117.717733,65.498908,20.057882,0.097473,7.671431,8.208187,0.091708,0.172444,5.245057,2.571120,6.642770,0.144252,2.278553,242.337767,0.155170,2.703431,12.063772,7.911342,5.010570,5.169194,54.870943,0.042715,11.206771,73.294443,54102.809452,0.044858,6.185045,0.159541,1.494564,0.567698,0.037877,0.073265,0.102933,0.041480,0.000000,0.000517,0.001140,60.380415,0.045829,30.995614,8044.140429,16.453345,16990.929460,8.195963,32.907237,2822.068430,0.111237,0.052984,0.019158,14297.468549,0.000061,0.123248,5.802480,143527.930743,1462.639333,0.100061,0.130513,264.532218,8.722339,0.004733,0.098317,7904.216646,0.000439,6.016849,3689.392510,359.204111,0.173498,0.162594,0.086385,48.126385,1.952486,1.952486,1287.813181,996.414899,379.227914,745.497289,6.356453
houses,484.642804,53.077859,115.894313,0.078349,6.698917,7.678433,0.084710,0.159251,4.438362,1.644704,10.820957,0.158350,0.167476,77.161481,0.150342,3.606794,9.284858,4.299074,8.316870,6.192786,104.773534,0.078200,21.441620,108.908007,110050.194059,0.033558,7.815975,0.151259,1.48

In [9]:

#

In [59]:
from core.utils import used_keys
used_keys['sdsAre']

'area covered by edge-attached ETCs'

In [60]:
from sklearn.preprocessing import StandardScaler

In [61]:
vals = StandardScaler().fit_transform(primary)
component_data = pd.DataFrame(vals, columns=primary.columns, index=primary.index)
component_data = component_data.drop(component_data.columns[component_data.std() == 0], axis=1)
vals = np.nan_to_num(component_data)
standardised_data = pd.DataFrame(vals, columns=component_data.columns, index=component_data.index)

In [62]:
standardised_data.describe().iloc[3:].style.background_gradient(axis=1, cmap="BuGn")

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,ssbCCM,ssbCCD,stbOri,mtbSWR,libNCo,ldbPWL,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbCeA,stbSAl,sdsLen,sssLin,ldsMSL,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,sddAre,midRea,midAre,stcOri,sdcLAL,sdcAre,sscCCo,sscERI,mtcWNe,mdcAre,ltcWRB,sicCAR,stcSAl,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB,sdsAre,likWCe,mibCou,mibAre,mibLen,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,licBAD,misBAD,midBAD
min,-0.259670,-1.011598,-0.016418,-4.882249,-1.411938,-0.402554,-10.479227,-3.226904,-1.264474,-0.612281,-1.542945,-0.483153,-0.087601,-0.539844,-4.596809,-1.095229,-1.286834,-2.301084,-0.869316,-0.933022,-0.614545,-6.380922,-1.227209,-1.582857,-1.016135,-0.542417,-3.239543,-3.208851,-2.541118,-2.066614,-18.600143,-6.606728,-1.293118,-1.750445,-1.405253,-0.999524,-1.943432,-0.637611,-0.552011,-0.761955,-0.578482,-1.116349,-0.781092,-1.617151,-1.412365,-0.550890,-3.277565,-9.388894,-0.168625,-0.892465,-0.001710,-1.107959,-1.048042,-0.336732,-0.444564,-2.431011,-2.381415,-0.664974,-1.567565,-0.023326,-1.157834,-0.505108,-0.001702,-0.283753,-0.255618,-0.396477,-2.709866,-5.595553,-3.463556,-0.709674,-1.407719,-1.407719,-0.240081,-0.380486,-0.272149,-0.280551
25%,-0.189163,-0.457710,-0.016418,-0.510067,-0.605687,-0.369763,-0.355934,-0.733440,-0.477507,-0.603048,-0.856024,-0.483153,-0.087601,-0.336862,-0.308896,-0.776489,-0.604045,-0.679639,-0.779273,-0.795979,-0.454676,-0.064145,-0.655973,-0.647083,-0.630252,-0.303878,-0.685550,-0.683673,-0.618475,0.192498,-0.717948,-0.517850,-0.696212,-0.672104,-0.835479,-0.449567,-0.865550,-0.496663,-0.552011,-0.497117,-0.481356,-0.589033,-0.553644,-0.873735,-0.624290,-0.431410,-0.648915,-0.476769,-0.070374,-0.596106,-0.001636,-0.723847,-0.829623,-0.315073,-0.372800,-0.794154,-0.833140,-0.570049,-0.875622,-0.017025,-0.683639,-0.441591,-0.001700,-0.283753,-0.220705,-0.267634,-0.732548,-0.202905,-0.664764,-0.427124,-0.520759,-0.520759,-0.199395,-0.308175,-0.210547,-0.216121
50%,-0.116310,-0.178540,-0.016418,0.235877,-0.202562,-0.320029,0.404741,0.099101,-0.181486,-0.214041,-0.089837,-0.483153,-0.087601,-0.234057,0.609943,-0.251901,-0.263328,-0.176209,-0.393876,-0.366742,-0.341322,0.443698,-0.430319,-0.191618,-0.379524,-0.138308,-0.185006,0.009779,0.088889,0.192498,-0.112182,0.078695,-0.213607,-0.088002,-0.239562,-0.012218,-0.033312,-0.264433,-0.552011,-0.371117,-0.381422,-0.200485,-0.371089,-0.061439,-0.346089,-0.330979,0.089555,0.218234,-0.010139,-0.362694,-0.001565,-0.236912,-0.315430,-0.240458,-0.255780,-0.082099,0.090878,-0.319687,-0.086190,-0.010266,-0.270344,-0.355835,-0.001699,-0.283753,-0.187308,-0.202295,0.109686,0.360787,0.254919,-0.287373,-0.163680,-0.163680,-0.172726,-0.260419,-0.174919,-0.179970
75%,-0.030079,0.117848,-0.016418,0.769760,0.200564,-0.180684,0.688342,0.828260,0.114872,0.205497,0.810863,0.000000,-0.087601,-0.044996,0.609943,0.489138,0.268449,0.481291,0.457831,0.494367,-0.079083,0.531605,0.308689,0.388065,0.254494,0.045275,0.515628,0.804978,0.713540,0.192498,0.590506,0.641936,0.460753,0.551399,0.641643,0.325212,0.725827,0.116161,0.641329,-0.013065,-0.013874,0.271324,0.106207,0.846834,0.297555,-0.052901,0.730385,0.659703,0.046720,0.238450,-0.001437,0.380643,0.586067,-0.050960,0.005947,0.739725,0.972545,0.185956,0.843041,0.007521,0.394111,-0.049601,-0.001695,-0.097535,-0.124975,-0.081648,0.830119,0.657401,0.814485,-0.020646,0.162224,0.162224,-0.102999,-0.067168,-0.077852,-0.080181
max,133.400991,64.564385,380.773640,4.437919,70.545952,13.969736,2.321142,1.523169,56.374560,77.148888,1.976792,5.067527,27.894444,32.040172,0.609943,7.093097,30.027599,14.759621,3.853762,3.711667,15.588680,0.532512,14.722484,10.814080,8.217303,41.054827,2.839241,1.601011,3.926073,3.581167,11.625128,2.907143,6.599303,8.224212,9.536219,112.619717,24.045947,36.073968,17.348093,28.224594,9.900237,15.063916,12.695368,1.864865,24.306713,76.818172,4.035688,2.507178,654.570687,25.007

In [63]:
# etcs.loc[morphotope_etcs[morphotope_etcs.types == 'houses'].index].explore(perfer_canvas=True, bassemap='cartodbpositron')

In [ ]:
def morphotopes_to_etcs(region_id, etcs=True, model_params='_100_0_None_None_False'):


    if etcs:
        etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/tessellations/tessellation_{region_id}.parquet')

    else:
        etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/buildings/buildings_{region_id}.parquet')

    etcs['label'] = -1

    morphotopes = pd.read_parquet(f'/data/uscuni-ulce/processed_data/morphotopes/tessellation_labels_morphotopes_{region_id}{model_params}.pq')
    morphotopes.loc[:, 'morphotope_label'] =  morphotopes.values[:, 0]

    morph_dict = pd.Series(np.arange(np.unique(morphotopes.values).shape[0]),
                       np.unique(morphotopes.values))
    etcs.loc[morphotopes.index, 'label'] = morphotopes.map(lambda x: morph_dict.loc[x]).values
    etcs['morph'] = str(region_id) + '_' + '-1'
    etcs.loc[morphotopes.index, 'morph'] = str(region_id) + '_' + morphotopes.values
    return etcs

In [65]:
etcs = morphotopes_to_etcs(region_id)

In [66]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.SolidPolygonLayer.from_geopandas(etcs, opacity=.08)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 6.73 s, sys: 374 ms, total: 7.1 s
Wall time: 7.1 s


In [67]:
from sidecar import Sidecar
sc = Sidecar(title='Final Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [68]:
from core.cluster_validation import get_color
layer.get_fill_color = get_color(etcs.label)

In [20]:
etcs['barea'] = buildings.area
etcs['barea'] = etcs['barea'].fillna(0)

In [21]:
street_graph = mm.gdf_to_nx(streets, preserve_index=True)
nodes, edges = mm.nx_to_gdf(street_graph, spatial_weights=False)


In [22]:
blg_nid = mm.get_nearest_street(
    buildings, edges
)

In [23]:
# %%time
# links = mm.get_network_ratio(etcs, edges)
# etcs[['edgeID_keys', 'edgeID_values']] = links
# keys = etcs.edgeID_values.apply(lambda a: np.argmax(a))
# tess_nid = np.array([inds[i] for inds, i in zip(etcs.edgeID_keys, keys)])

# etc_nid = pd.Series(tess_nid, index=etcs.index)

In [24]:
etc_nid = mm.get_nearest_street(
    etcs, edges
)

In [25]:
%%time
inpidxs, tree_idxs = streets.sindex.query(etcs.geometry, predicate='dwithin', distance=1)

CPU times: user 3.87 s, sys: 1.01 ms, total: 3.87 s
Wall time: 3.87 s


In [26]:
street_etcarea_std = etcs.iloc[inpidxs].area.groupby(streets.index[tree_idxs]).std()
street_etcarea_std = street_etcarea_std.fillna(0)
etc_street_std = etc_nid.map(lambda x: street_etcarea_std.loc[x] if x in street_etcarea_std else 0)

In [27]:
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
from libpysal.graph import read_parquet, Graph

graph = read_parquet(graph_dir + f'tessellation_graph_{region_id}.parquet')

In [ ]:
# graph = Graph.build_fuzzy_contiguity(etcs.geometry, tolerance=.1)

In [29]:
graph3 = graph.higher_order(k=3, lower_order=True, diagonal=True)

In [ ]:
# differences between apartment blocks and industrial -
# ssbEri (more rectangular),
# ssbCCD ( more complex patterns),
# sdsLen ( simpler streets),
# ldsMSL(longer, simpler streets),
# ldsRea(more buildings),
# ldsAre(more built up area),
# ldsCDL(simpler streets),
# mtdMDi(simpler streets),
# sddAre(more built up area),
# midRea(more buildings),
# sdsAre(more built up area)



# stds
# sdbAre(less regular patterns) - std in etc neighbours, - seems to work - barea_std, baarea3_std

# sssLin(less complex streets) - std in etc neighbours, - meh

# sdsAre (less regular patterns& more built up), - meh

# sddAre (less regular patterns & more built up), - difficult

# midRea	(less regular patterns& more built up),
# midAre (less regular patterns & more built up),

# ldsAre (more built up) - std in large 3 neighbours,
# ldsCDL(less regular streets) - std in large 3 neighbours,

In [31]:
## variety stuff

In [32]:
# barea-stds - good
etcs['barea'] = buildings.area
etcs['barea'] = etcs['barea'].fillna(0)
assert (etcs.loc[buildings.index, 'barea'] == buildings.area).all()

res = graph.describe(etcs['barea'])
etcs['micBAD'] = res['std']

In [33]:
res = graph3.describe(etcs['barea'])
etcs['barea3_std'] = res['std']

In [34]:
# variability in street attached buildings area
street_barea_std = buildings.area.groupby(blg_nid).std()
street_barea_std = street_barea_std.fillna(0)
blg_street_std = blg_nid.map(lambda x: street_barea_std.loc[x])

etcs['blg_street_std'] = blg_street_std
etcs['blg_street_std'] = etcs['blg_street_std'].fillna(0)

In [35]:
etcs['blg_street_std_3'] = graph3.describe(etcs['blg_street_std'])['mean']

In [36]:
# variability in node attached buildings area

bnode_id = mm.get_nearest_node(
    buildings, nodes, edges,  blg_nid
)


node_barea_std = buildings.area.groupby(bnode_id).std()
node_barea_std = node_barea_std.fillna(0)
blg_node_std = bnode_id.map(lambda x: node_barea_std.loc[x])

etcs['blg_node_std'] = blg_node_std
etcs['blg_node_std'] = etcs['blg_node_std'].fillna(0)

etcs['blg_node_std_3'] = graph3.describe(etcs['blg_node_std'])['mean']

In [37]:
# # variability in street attached built up area - meh
etcs['etc_street_std'] = graph.describe(primary['sdsAre'])['std'].fillna(0)
etcs['etc_street3_std'] = graph3.describe(primary['sdsAre'])['std'].fillna(0)

In [38]:
# linearity in different ways - meh
etcs['sssLin_std'] = graph.describe(primary['sssLin'])['std']
etcs['sssLin3_std'] = graph3.describe(primary['sssLin'])['std']

In [39]:
# cell area variability - meh
res = graph.describe(tess['sdcAre'])
etcs['tarea_std'] = res['std']

# cell area variabilit inside enclosure - meh
enc_etcarea_std = etcs.area.groupby(etcs['enclosure_index']).std().fillna(0)
etcs['enc_etcarea_std'] = etcs['enclosure_index'].map(lambda x: enc_etcarea_std.loc[x])

In [40]:
# ## doesnt really add much. mean is basically area
# etcs['cor_std'] = res['std']
# etcs['cor_mean'] = res['mean']

# etcs['cor_std'] = etcs['cor_std'].fillna(0)
# etcs['cor_mean'] = etcs['cor_mean'].fillna(0)

# res = graph.describe(etcs['cor_std'])
# etcs['cor_std'] = res['mean']

# res = graph.describe(etcs['cor_mean'])
# etcs['cor_mean'] = res['std']

In [41]:
## nothing much
etcs['sicar'] = etcs['barea'] / etcs.area
res = graph.describe(etcs['sicar'])
etcs['sicar_std'] = res['mean']

In [42]:
etcs['bperi'] = buildings.length
etcs['bperi'] = etcs['bperi'].fillna(0)
res = graph.describe(etcs['bperi'])
etcs['bperi_std'] = res['std']

In [43]:
import momepy as mm

In [44]:
etcs['fr'] = mm.facade_ratio(etcs)

In [45]:
res = graph.describe(etcs['fr'])
etcs['fr_std'] = res['std']

In [46]:
enc_etcarea_std = etcs.area.groupby(etcs['enclosure_index']).std().fillna(0)
etcs['enc_etcarea_std'] = etcs['enclosure_index'].map(lambda x: enc_etcarea_std.loc[x])

In [47]:
morphotope_etcs = etcs.loc[morphotopes.index]
morphotope_etcs['mlabel'] = morphotopes.values

types = pd.Series([0,0, 1,1, 2,2,2,2, 3,3,  4,4], index=[134,14, 18,31, 7,5,37,36, 57,58, 30,28])
types = types.replace({0: 'houses', 1: 'grid', 2: 'apartment blocks', 3: 'industry', 4: 'medieval'})
morphotope_etcs = morphotope_etcs[morphotope_etcs['mlabel'].isin(types.index)]
morphotope_etcs['types'] = morphotope_etcs['mlabel'].map(lambda x: types.loc[x])

In [48]:
res = morphotope_etcs.iloc[:, 4:-1].groupby(morphotope_etcs['types']).mean().loc[['industry', 'apartment blocks', 'medieval', 'grid', 'houses'],]
res.style.background_gradient(axis=0, cmap="BuGn")

,barea,micBAD,barea3_std,blg_street_std,blg_street_std_3,blg_node_std,blg_node_std_3,etc_street_std,etc_street3_std,sssLin_std,sssLin3_std,tarea_std,enc_etcarea_std,sicar,sicar_std,bperi,bperi_std,fr,fr_std,mlabel
types,,,,,,,,,,,,,,,,,,,,
industry,1270.474295,2634.168230,3059.236459,2305.216863,2248.122462,2264.983961,2188.504263,35734.614655,72703.019436,0.036655,0.066737,6741.500784,6692.124425,0.195775,0.207500,127.329200,130.558304,14.744596,7.813855,57.739766
apartment blocks,1512.320135,1407.954667,1643.761618,635.374680,570.255263,648.439996,587.165378,23761.587770,31590.877663,0.110462,0.159199,7786.640832,6673.825234,0.157676,0.153828,230.072690,165.844452,21.723473,6.865555,17.415512
medieval,480.659535,521.189245,724.441290,372.506411,411.013166,382.621537,410.596430,6417.238406,11057.477929,0.044930,0.078155,1273.823821,2403.058011,0.547249,0.536848,118.202184,85.981864,6.304150,3.231142,29.269841
grid,403.484888,413.879422,643.952177,198.819797,248.297276,305.698210,338.177197,4831.723008,8014.616595,0.011663,0.022796,1768.257908,1072.805078,0.424163,0.415892,83.810831,43.217808,6.815800,2.961099,25.874862
houses,205.579261,224.319291,371.246612,158.573021,168.084916,199.141506,194.545158,4683.367742,7785.806891,0.039183,0.064485,1194.955707,887.382979,0.168002,0.172995,56.868680,33.107223,7.342753,2.779598,52.270270


In [ ]:
# barea_stds and blg_street_stds, etc_street3_std, sssLin,

In [ ]:
### fixed minimum for buildings and tess area to limit all the weird variability...

In [134]:
import lonboard
from sidecar import Sidecar

In [135]:
plotting = etcs[etcs.index >= 0]

In [136]:
plotting.loc[:, 'barea'] = plotting.barea.clip(1, ).fillna(0)
plotting.loc[:, 'barea_std'] = plotting.barea_std.clip(1, )

In [126]:
layer = lonboard.SolidPolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [127]:
sc = Sidecar(title='tess')
m = lonboard.Map(layer)
with sc:
    display(m)

In [128]:
etcs.columns

Index(['enclosure_index', 'geometry', 'label', 'morph', 'barea', 'sssLin_std',
       'sssLin3_std', 'barea_std', 'barea3_std', 'tarea_std',
       'enc_etcarea_std', 'blg_street_std', 'etc_street_std', 'cor_std',
       'cor_mean', 'sicar', 'sicar_std', 'bperi', 'bperi_std', 'fr', 'fr_std',
       'sssLin_std3', 'blg_street3_std'],
      dtype='object')

In [156]:
column = 'blg_street3_std'

In [157]:
# barea_std, barea3_std

In [169]:
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify

classifier = classify(plotting[column], 'quantiles', k=100)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/mapclassify/classifiers.py:1653: UserWarning: Not enough unique values in array to form 100 classes. Setting k to 95.
  self.bins = quantile(y, k=k)


In [ ]:
from core.cluster_validation import get_color
a = np.log(plotting[column])
# quintiles = np.nanpercentile(a, np.arange(1, 100, 1).tolist() + [99.5, 99.9])
quintiles = np.nanpercentile(a, np.arange(1, 100, 1).tolist())
vals = np.searchsorted(quintiles, a)
layer.get_fill_color = get_color(vals)

In [63]:
locs = [445777, 445835, 445963]

In [79]:
# etcs.loc[locs].explore()

In [66]:
blg_nid.loc[locs]

445777    74418.0
445835    74417.0
445963    47646.0
dtype: float64